In [1]:
from config.global_config import GlobalConfig
from pipeline import Pipeline
from training import AnnabellPreTrainingTestingRunner, AnnabellTestingRunner

global_config = GlobalConfig()

2025-11-26 06:50:20,083 - root - INFO - Logging initialized. Log file: /Users/chris/logs/cognitive_language_model_logs/run_20251126_065020.log


In [2]:

pipeline = Pipeline(prepared_dataset_filepath=global_config.prepared_dataset_with_commands_filepath())
pipeline.load_prepared_dataset()
#pipeline.run_pre_training_evaluation_testing()
#pipeline.run_evaluate_pre_training_results()
#runner = AnnabellPreTrainingTestingRunner(pipeline.dataset_processor)
#runner.move_annabell_logfile_to_gdrive()

2025-11-25 06:32:35,797 - pipeline - INFO - Loading prepared dataset from /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/sandbox/data_sets/nyc_squad_with_pretraining_commands.jsonl...
2025-11-25 06:32:36,709 - pipeline - INFO - Prepared dataset loaded successfully.


# todo

1. only add prw if the wg is not the last one.

.ph a golden statue of the_Virgin_Mary sit on top of the_Main_Building
.wg a golden statue
.prw
.wg of the_Virgin_Mary
.prw
.rw

2. why .ph is written twice?
#id: 5733be284776f41900661180
the Basilica of the Sacred Heart at Notre_Dame be beside
the_Main_Building


? the Basilica of the sacred heart at Notre_Dame be
beside to which structure
.sctx ? the Basilica of the sacred heart at Notre_Dame be
.wg Basilica
.ph the Basilica of the Sacred Heart at Notre_Dame be beside
.ph the Basilica of the Sacred Heart at Notre_Dame be beside
.wg Notre_Dame
.sctx beside to which structure
.wg beside
.ph the Basilica of the Sacred Heart at Notre_Dame be beside
.ph the Basilica of the Sacred Heart at Notre_Dame be beside
.sctx the_Main_Building
.wg the_Main_Building
.rw

3. associations only built up to max words -1 in the below example "of prayer" wg is not built on phrase ingestion
#id: 5733be284776f41900661181
the Grotto at Notre_Dame be a marian place of prayer
and reflection


? what be the Grotto at Notre_Dame
.sctx ? what be the Grotto at Notre_Dame
.wg Grotto
.wg Notre_Dame
.ph the Grotto at Notre_Dame be a marian place of prayer
.wg a marian place
.prw
.wg of prayer
.prw
.sctx and reflection
.wg and reflection
.rw

In [2]:
from training import AnnabellPreTrainingTestingRunner

pipeline = Pipeline(prepared_dataset_filepath=global_config.prepared_dataset_with_commands_filepath())
pipeline.load_prepared_dataset()
runner = AnnabellTestingRunner(pipeline.dataset_processor)

pipeline.run_evaluate_training_results()

2025-11-26 06:50:32,848 - pipeline - INFO - Loading prepared dataset from /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/sandbox/data_sets/nyc_squad_with_pretraining_commands.jsonl...
2025-11-26 06:50:33,759 - pipeline - INFO - Prepared dataset loaded successfully.
2025-11-26 06:50:33,760 - pipeline - INFO - Starting evaluation of training results...
2025-11-26 06:50:33,762 - testing - INFO - length of log file questions and answers: 2
2025-11-26 06:50:33,764 - testing - INFO - number of test samples not found in training data: 0
2025-11-26 06:50:33,765 - testing - INFO - test samples not found in training data: [] ...


Generating test answer embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-26 06:50:34,225 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-26 06:50:34,321 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings: 100%|██████████| 1/1 [00:00<00:00, 10.93it/s]

2025-11-26 06:50:34,327 - testing - INFO - number of test answers longer than 20 words: 0
2025-11-26 06:50:34,328 - testing - INFO - number correct = 0 out of 1
2025-11-26 06:50:34,329 - testing - INFO - percentage correct = 0.0 %
2025-11-26 06:50:34,332 - testing - INFO - number any word matches = 0 out of 1
2025-11-26 06:50:34,332 - testing - INFO - percentage any word matches = 0.0 %
2025-11-26 06:50:34,333 - testing - INFO - number of rows with cosine distance less than 0.1: 0
2025-11-26 06:50:34,333 - testing - INFO - percentage of total: <bound method AnnabellTestResultsEvaluator.percentage_of_answers_below_cosine_distance_threshold of <testing.AnnabellTestResultsEvaluator object at 0x137ad0830>> %
2025-11-26 06:50:34,333 - testing - INFO - number of rows with cosine distance less than 0.1 and any matching answer correct: 0
percentage of total: 0.0 %
2025-11-26 06:50:34,336 - testing - INFO - total number of test samples in input file: 2
results written to /Users/chris/Library/Cl

In [2]:
global_config.prepared_dataset_with_commands_filepath_exists()

True

In [3]:
from testing import AnnabellPreTrainingTestContext, AnnabellTestResultsEvaluator

pipeline = Pipeline(prepared_dataset_filepath=global_config.prepared_dataset_with_commands_filepath())
pipeline.load_prepared_dataset()
testing_context = AnnabellPreTrainingTestContext(pipeline.datasetPreProcessor)
evaluator = AnnabellTestResultsEvaluator(testing_context)
evaluator.run()

2025-11-24 14:16:49,897 - pipeline - INFO - Loading prepared dataset from /home/chris/gdrive/work/annabell/experiments/sandbox/data_sets/nyc_squad_with_pretraining_commands.jsonl...
2025-11-24 14:16:52,110 - pipeline - INFO - Prepared dataset loaded successfully.
2025-11-24 14:16:53,180 - testing - INFO - length of log file questions and answers: 4
2025-11-24 14:16:53,182 - testing - INFO - number of test samples not found in training data: 0
2025-11-24 14:16:53,183 - testing - INFO - test samples not found in training data: [] ...


Generating test answer embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

2025-11-24 14:16:53,951 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating test answer embeddings:  67%|██████▋   | 2/3 [00:00<00:00,  2.61it/s]

2025-11-24 14:16:54,040 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-24 14:16:54,129 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

2025-11-24 14:16:54,222 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-11-24 14:16:54,306 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings: 100%|██████████| 3/3 [00:00<00:00, 17.07it/s]

2025-11-24 14:16:54,397 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Generating response answer embeddings: 100%|██████████| 3/3 [00:00<00:00, 11.24it/s]


2025-11-24 14:16:58,196 - testing - INFO - number of test answers longer than 20 words: 0
2025-11-24 14:16:58,197 - testing - INFO - number correct = 2 out of 3
2025-11-24 14:16:58,198 - testing - INFO - percentage correct = 66.66666666666666 %
2025-11-24 14:16:58,199 - testing - INFO - number any word matches = 2 out of 3
2025-11-24 14:16:58,200 - testing - INFO - percentage any word matches = 66.66666666666666 %
2025-11-24 14:16:58,200 - testing - INFO - number of rows with cosine distance less than 0.1: 2
2025-11-24 14:16:58,201 - testing - INFO - percentage of total: <bound method AnnabellTestResultsEvaluator.percentage_of_answers_below_cosine_distance_threshold of <testing.AnnabellTestResultsEvaluator object at 0x7ff8ccbdffe0>> %
2025-11-24 14:16:58,201 - testing - INFO - number of rows with cosine distance less than 0.1 and any matching answer correct: 2
percentage of total: 66.66666666666666 %
2025-11-24 14:17:01,168 - testing - INFO - total number of test samples in input file:

In [ ]:
import spacy
from spacy.cli import download


def _load_spacy_model(model_name):
    """Loads a spaCy model, downloading it if necessary."""
    try:
        return spacy.load(model_name)
    except OSError:
        download(model_name)
        return spacy.load(model_name)


# Load the small English model
nlp = _load_spacy_model("en_core_web_sm")

In [3]:

# The sentence (adjusting for the standard space usage)
text = "what sit on top of the Main_Building at Notre_Dame"

# Process the text
doc = nlp(text)

print("--- Token and Dependency Analysis ---")
print("{:<10} {:<10} {:<10} {:<10}".format("TOKEN", "LEMMA", "DEP", "HEAD"))
print("-" * 40)
for token in doc:
    # DEP: The dependency relation label
    # HEAD: The head token of this token
    print("{:<10} {:<10} {:<10} {:<10}".format(
        token.text, token.lemma_, token.dep_, token.head.text
    ))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 785.8 kB/s  0:00:160:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
--- Token and Dependency Analysis ---
TOKEN      LEMMA      DEP        HEAD      
----------------------------------------
what       what       nsubj      sit       
sit        sit        ROOT       sit       
on         on         prep       sit       
top        top        pobj       on        
of         of         prep       top       
the        the        det        Main_Building
Main_Building main_building pobj       of        
at         at         prep       sit       
Notre_Dame Notre_Dame pobj       at        


In [4]:
from spacy import displacy

displacy.serve(doc, style="dep")

ValueError: [E1050] Port 5000 is already in use. Please specify an available port with `displacy.serve(doc, port=port)` or use `auto_select_port=True` to pick an available port automatically.